In [1]:
import keras

In [2]:
from keras.datasets import cifar10
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten,AveragePooling2D,Dropout,BatchNormalization,Activation
from keras.models import Model,Input
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
import os

In [3]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [4]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [5]:
print('Train Images',x_train.shape)
print('Train Labels',y_train.shape)
print('Test Images',x_test.shape)
print('Test Labels',y_test.shape)

Train Images (50000, 32, 32, 3)
Train Labels (50000, 1)
Test Images (10000, 32, 32, 3)
Test Labels (10000, 1)


In [6]:
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

In [7]:
def MiniModel(input_shape):
  images = Input(input_shape)

  net = BatchNormalization()(images)
  net = Activation('relu')(net)
  net = Conv2D(filters=64,kernel_size=[3,3],strides=[1,1],padding='same')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters=64,kernel_size=[3,3],strides=[1,1],padding='same')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters=64,kernel_size=[3,3],strides=[1,1],padding='same',activation='relu')(net)
  net = MaxPool2D(pool_size=(2,2))(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters=128,kernel_size=[3,3],strides=[1,1],padding='same',activation='relu')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters=128,kernel_size=[3,3],strides=[1,1],padding='same',activation='relu')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters=128,kernel_size=[3,3],strides=[1,1],padding='same',activation='relu')(net)
  net = MaxPool2D(pool_size=(2,2))(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters=256,kernel_size=[3,3],strides=[1,1],padding='same',activation='relu')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters=256,kernel_size=[3,3],strides=[1,1],padding='same',activation='relu')(net)

  net = BatchNormalization()(net)
  net = Activation('relu')(net)
  net = Conv2D(filters=256,kernel_size=[3,3],strides=[1,1],padding='same',activation='relu')(net)


  net = Dropout(0.25)(net)
  net = AveragePooling2D(pool_size=(8,8))(net)
  net = Flatten()(net)
  net = Dense(units=10,activation='softmax')(net)

  model = Model(inputs = images, outputs=net)

  return model


In [8]:
input_shape = (32,32,3)
model = MiniModel(input_shape)


model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
activation (Activation)      (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)       

In [9]:
def lr_schedule(epoch):
  lr = 0.001
  if epoch > 15:
    lr = lr/100
  elif epoch > 10:
    lr = lr/10
  elif epoch > 5:
    lr = lr / 5
  print("Learning rate: ",lr)
  return lr

In [10]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [11]:
save_dir = os.path.join(os.getcwd(), 'cifar10savemodels')

model_name = 'cifar10model.{epoch:03d}.h5'

In [12]:
if not os.path.join(save_dir):
  os.makedirs(save_dir)

modelpath = os.path.join(save_dir,model_name)

checkpoint = ModelCheckpoint(filepath= modelpath,
                             monitor='val_acc',
                             verbose = 1,
                             save_best_only = True,
                             period = 1)

In [13]:
model.compile(optimizer=Adam(lr_schedule(0)),loss='categorical_crossentropy',metrics=['accuracy'])

Learning rate:  0.001


In [14]:
model.fit(x_train,y_train,batch_size=128,epochs=20,shuffle=True,validation_split=0.1,verbose=1,callbacks=[checkpoint,lr_scheduler])

accuracy = model.evaluate(x=x_test,y=y_test,batch_size=128)
print("Accuracy", accuracy[1])

Learning rate:  0.001
Epoch 1/20
352/352 [==============================] - 17s 49ms/step - loss: 1.5305 - accuracy: 0.4475 - val_loss: 2.1052 - val_accuracy: 0.3052
Learning rate:  0.001
Epoch 2/20
352/352 [==============================] - 17s 48ms/step - loss: 1.0091 - accuracy: 0.6426 - val_loss: 1.3853 - val_accuracy: 0.5566
Learning rate:  0.001
Epoch 3/20
352/352 [==============================] - 17s 48ms/step - loss: 0.6989 - accuracy: 0.7576 - val_loss: 0.9752 - val_accuracy: 0.6734
Learning rate:  0.001
Epoch 4/20
352/352 [==============================] - 17s 47ms/step - loss: 0.5173 - accuracy: 0.8193 - val_loss: 0.8181 - val_accuracy: 0.7308
Learning rate:  0.001
Epoch 5/20
352/352 [==============================] - 17s 47ms/step - loss: 0.3996 - accuracy: 0.8597 - val_loss: 0.6620 - val_accuracy: 0.7790
Learning rate:  0.001
Epoch 6/20
352/352 [==============================] - 17s 47ms/step - loss: 0.3091 - accuracy: 0.8911 - val_loss: 0.5735 - val_accuracy: 0.8070
Lear